In [1]:
import os
import imaplib
import smtplib
import email
import re
from email.message import EmailMessage
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableBranch, RunnableLambda
from database import store_email_result, get_latest_thread_state, store_admin_escalation
from firebase_admin import firestore
from database import store_email_result, update_email_state

In [2]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
classification_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an HR assistant. Use both the current email and previous context to classify the email as one of the following: Leave Request, Job Inquiry, Onboarding. If you are not able to categorize it as above three categorize it as Escalate."),
    ("human", "Previous conversation (if any):\n{history}\n\nCurrent email:\n{email}")
])


In [4]:
classification_chain = classification_prompt | model | StrOutputParser()

In [12]:
classification_chain.invoke({
            "email": "I want to take leave for 2 weeks",
            "history": ""
        }).strip().lower().replace(" ", "_")

'category:_leave_request'

In [16]:
load_dotenv()
EMAIL_USER = os.getenv("EMAIL_ADDRESS")
print(EMAIL_USER)

hairagent88@gmail.com
